## Merlin6 adaptations

On PSI's Merlin6 cluster, this notebook should be run from the GPU queue (launch jupyter with `-p gpu --gres=gpu:1 --account=merlin-gpu`). Then, go to Kernel » Change Kernel » Python \[conda env:cryocare\].

The following changes have been made to this notebook for Merlin6 compatibility:

- Data is stored in ./data rather than /data
- Example data is downloaded automatically
- MotionCor2 is called using the correct path (MotionCor2_1.1.0-Cuda91) and cuda version. The environment does not need to be changed (it is configured correctly in the jupyter kernel)
- Fix small bugs
    - header -p returns a single line (so access as `p[0]`)

In [1]:
!hostname

merlin-g-001.psi.ch


In [2]:
# Test that MotionCor2 was loaded
!MotionCor2_1.1.0-Cuda91 -h


Usage: MotionCor2 Tags
-InMrc         
  1. Input MRC file that stores dose fractionated stacks.
  2. It can be a MRC file containing a single stack collected
     in Leginon or multiple stacks collected in UcsfTomo.
  3. It can also be the path of a folder containing multiple
     MRC files when -Serial option is turned on.

-InTiff        
  1. Input TIFF file that stores a dose fractionated stack.
-OutMrc        
  1. Output MRC file that stores the frame sum.
  2. It can be either a MRC file name or the prefix of a series
     MRC files when -Serial option is turned on.

-ArcDir        
 1. Path of the archive folder that holds the archived raw
    stacks with each pixel packed into 4 bits.
 2. The archived stacks are saved in MRC file with the gain
    reference saved in the extended header.
 3. The rotated and/or flipped gain reference will be saved
    if -RotGain and or -FlipGain are enabled.

-FullSum       
 1. MRC file for global-motion corrected, unweighted sum.
 2. This f

# Split Movie Frames

The first step in training a cryo-CARE network is to split the direct detector acquisition movies. To that end we will first align the frames with [MotionCor2](https://www.nature.com/articles/nmeth.4193) and then split the frames into even and odd acquistions.

__Note__: This only works if you have downloaded MotionCor2 and placed it in the `example` directory. For more details check out at the [README](https://github.com/juglab/cryoCARE_simg/blob/master/README.md#get-motioncor2).

In [3]:
from glob import glob
import mrcfile
import os
from os.path import join, basename

import numpy as np
from tqdm import tqdm

from matplotlib import pyplot as plt

from utils import *

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
%%bash
# Download example data (9GB)
if ! [ -d 'data/Tomo110' ]; then
    [ -e 'data/Tomo110.zip' ] ||
        curl -o 'data/Tomo110.zip' 'https://cloud.mpi-cbg.de/index.php/s/prTOcYsFfPNa1mG/download'
    unzip -d 'data/' 'data/Tomo110.zip'
fi

In [5]:
# In this example we use the downloaded tomogram 'Tomo110', 
# kindly provided by Mareike Jordan from the Pigino Lab at 
# MPI-CBG, Dresden. 
data_path = 'data/Tomo110/frames/'

In [6]:
# Read the list of mrc-files and display them.
files = glob(data_path + '*.mrc')
files

['data/Tomo110/frames/31_Tomo110_4.0_Apr04_15.29.05.mrc',
 'data/Tomo110/frames/58_Tomo110_-50.0_Apr04_16.04.17.mrc',
 'data/Tomo110/frames/04_Tomo110_58.0_Apr04_15.48.58.mrc',
 'data/Tomo110/frames/28_Tomo110_10.0_Apr04_15.31.18.mrc',
 'data/Tomo110/frames/64_Tomo110_-62.0_Apr04_16.10.49.mrc',
 'data/Tomo110/frames/22_Tomo110_22.0_Apr04_15.35.40.mrc',
 'data/Tomo110/frames/45_Tomo110_-24.0_Apr04_15.54.44.mrc',
 'data/Tomo110/frames/54_Tomo110_-42.0_Apr04_16.01.22.mrc',
 'data/Tomo110/frames/42_Tomo110_-18.0_Apr04_15.21.05.mrc',
 'data/Tomo110/frames/38_Tomo110_-10.0_Apr04_15.24.02.mrc',
 'data/Tomo110/frames/60_Tomo110_-54.0_Apr04_16.05.46.mrc',
 'data/Tomo110/frames/09_Tomo110_48.0_Apr04_15.45.13.mrc',
 'data/Tomo110/frames/12_Tomo110_42.0_Apr04_15.42.59.mrc',
 'data/Tomo110/frames/59_Tomo110_-52.0_Apr04_16.05.01.mrc',
 'data/Tomo110/frames/03_Tomo110_60.0_Apr04_15.49.50.mrc',
 'data/Tomo110/frames/53_Tomo110_-40.0_Apr04_16.00.38.mrc',
 'data/Tomo110/frames/29_Tomo110_8.0_Apr04_15.30

In [7]:
# We can also execute imod-commands by putting a '!' infront of 
# the command.
!header {files[0]}


 RO image file on unit   1 : data/Tomo110/frames/31_Tomo110_4.0_Apr04_15.29.05.mrc     Size=     556211 K

 Number of columns, rows, sections .....    7420    7676      10
 Map mode ..............................    0   (byte)                     
 Start cols, rows, sects, grid x,y,z ...    0     0     0    7420   7676     10
 Pixel spacing (Angstroms)..............   2.355      2.355      2.355    
 Cell angles ...........................   90.000   90.000   90.000
 Fast, medium, slow axes ...............    X    Y    Z
 Origin on x,y,z .......................    0.000       0.000       0.000    
 Minimum density .......................   0.0000    
 Maximum density .......................   143.00    
 Mean density ..........................   5.3446    
 tilt angles (original,current) ........   0.0   0.0   0.0   0.0   0.0   0.0
 Space group,# extra bytes,idtype,lens .        0        0        0        0

     1 Titles :
SerialEMCCD: Dose frac. image, scaled by 16.00  r/f 1   04-Ap

In [8]:
# Here we read out the pixel spacing from the imod-command.
# Pixel spacing (Angstroms) according to the header
p = !header -p {files[0]}
pixel_spacing = float(p[1].split()[0])
print('Pixel Spacing in Angstroms:', pixel_spacing)

Pixel Spacing in Angstroms: 2.3555


## Running MotionCor2

Merlin6 is configured to use MotionCor2 with cuda 9.1. This differs slightly from the version in the CryoCARE singularity container.

The command `nvcc --version` shows which `CUDA` is currently active.

In [9]:
# Execute the nvcc command
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Nov__3_21:07:56_CDT_2017
Cuda compilation tools, release 9.1, V9.1.85


## Align & Split

Each file gets aligned with MotionCor2 and the aligned stack is written to a `tmp` directory. Then the aligned stack is loaded and splitted in the two halves (even/odd) and written to the corresponding directories.

In [10]:
# Create directories
if not os.path.isdir('tmp'):
    os.mkdir('tmp')
if not os.path.isdir(join(data_path, 'even')):
    os.mkdir(join(data_path, 'even'))
if not os.path.isdir(join(data_path, 'odd')):
    os.mkdir(join(data_path, 'odd'))

In [11]:
def align_and_split(files):
    for f in tqdm(files):
        # Align the frames with MotionCor2 and write out the aligned stack with '-OutStack 1'
        ! MotionCor2_1.1.0-Cuda91 -InMrc {f} -OutMrc tmp/aligned.mrc -Patch 5 5 5 -OutStack 1 >> motioncor2.log
        # Since we process image by image only one aligned stack is available here
        aligned_stack = mrcfile.open(glob('tmp/*_Stk.mrc')[0], permissive=True)
        # Save even frames
        save_mrc(join(data_path, 'even', basename(f)), np.sum(aligned_stack.data[::2], axis=0), pixel_spacing)
        # Save odd frames
        save_mrc(join(data_path, 'odd', basename(f)), np.sum(aligned_stack.data[1::2], axis=0), pixel_spacing)
        # Remove aligned files
        remove_files('tmp', extension='.mrc')

In [12]:
# MotionCor2 will create a log file 
align_and_split(files)

  0%|          | 0/65 [00:00<?, ?it/s]/opt/psi/Programming/anaconda/2019.07/conda/envs/cryocare/lib/python3.7/site-packages/mrcfile/mrcinterpreter.py:208: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)
  2%|▏         | 1/65 [00:10<10:48, 10.13s/it]/opt/psi/Programming/anaconda/2019.07/conda/envs/cryocare/lib/python3.7/site-packages/mrcfile/mrcinterpreter.py:208: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)
  3%|▎         | 2/65 [00:20<10:38, 10.13s/it]/opt/psi/Programming/anaconda/2019.07/conda/envs/cryocare/lib/python3.7/site-packages/mrcfile/mrcinterpreter.py:208: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)
  5%|▍         | 3/65 [00:29<10:18,  9.98s/it]/opt/psi/Programming/anaconda/2019.07/conda/envs/cryocare/lib/python3.7/site-packages/mrcfile/mrcinterpreter.py:208: RuntimeWarn

 46%|████▌     | 30/65 [05:00<05:51, 10.05s/it]/opt/psi/Programming/anaconda/2019.07/conda/envs/cryocare/lib/python3.7/site-packages/mrcfile/mrcinterpreter.py:208: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)
 48%|████▊     | 31/65 [05:10<05:39,  9.99s/it]/opt/psi/Programming/anaconda/2019.07/conda/envs/cryocare/lib/python3.7/site-packages/mrcfile/mrcinterpreter.py:208: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)
 49%|████▉     | 32/65 [05:19<05:26,  9.90s/it]/opt/psi/Programming/anaconda/2019.07/conda/envs/cryocare/lib/python3.7/site-packages/mrcfile/mrcinterpreter.py:208: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)
 51%|█████     | 33/65 [05:30<05:20, 10.01s/it]/opt/psi/Programming/anaconda/2019.07/conda/envs/cryocare/lib/python3.7/site-packages/mrcfile/mrcinterpreter.py:208:

 92%|█████████▏| 60/65 [10:05<00:50, 10.07s/it]/opt/psi/Programming/anaconda/2019.07/conda/envs/cryocare/lib/python3.7/site-packages/mrcfile/mrcinterpreter.py:208: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)
 94%|█████████▍| 61/65 [10:15<00:40, 10.04s/it]/opt/psi/Programming/anaconda/2019.07/conda/envs/cryocare/lib/python3.7/site-packages/mrcfile/mrcinterpreter.py:208: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)
 95%|█████████▌| 62/65 [10:25<00:29,  9.93s/it]/opt/psi/Programming/anaconda/2019.07/conda/envs/cryocare/lib/python3.7/site-packages/mrcfile/mrcinterpreter.py:208: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)
 97%|█████████▋| 63/65 [10:35<00:19,  9.96s/it]/opt/psi/Programming/anaconda/2019.07/conda/envs/cryocare/lib/python3.7/site-packages/mrcfile/mrcinterpreter.py:208:

In [13]:
!cat motioncor2.log


-InMrc           data/Tomo110/frames/31_Tomo110_4.0_Apr04_15.29.05.mrc
-InTiff          
-InSuffix        
-OutMrc          tmp/aligned.mrc
-ArcDir          
-FullSum         
-Gain            
-Dark            
-DefectFile      
-DefectMap       
-InAln           
-OutAln          
-TmpFile         
-LogFile         
-Serial          0
-Patch           5  5  5
-Iter            7
-Tol             0.50
-Bft             500.00 150.00
-PhaseOnly       0
-StackZ          0
-FtBin           1.00
-InitDose        0.00
-FmDose          0.00
-PixSize         0.00
-kV              300
-Throw           0
-Trunc           0
-Group           1
-FmRef           -1
-OutStack        1
-RotGain         0
-FlipGain        0
-Align           1
-Tilt            0.00  0.00
-Mag             1.00  1.00  0.00
-InFmMotion      0
-Crop            0  0
-Gpu             0
-GpuMemUsage     0.50


GPU 0 memory: 8117 MB

Try to Load gain reference from MRC file.
CLoadRefs: load gain reference from header.
Info: In

Iteration  3  Error:     0.239

Align patch 18  7 left
Iteration  1  Error:     2.055
Iteration  2  Error:     0.735
Iteration  3  Error:     0.086

Align patch 19  6 left
Iteration  1  Error:     4.664
Iteration  2  Error:     1.965
Iteration  3  Error:     0.925
Iteration  4  Error:     0.408

Align patch 20  5 left
Iteration  1  Error:     9.398
Iteration  2  Error:     0.691
Iteration  3  Error:     0.066

Align patch 21  4 left
Iteration  1  Error:     6.929
Iteration  2  Error:     1.340
Iteration  3  Error:     0.352

Align patch 22  3 left
Iteration  1  Error:     2.063
Iteration  2  Error:     0.908
Iteration  3  Error:     0.191

Align patch 23  2 left
Iteration  1  Error:     3.057
Iteration  2  Error:     0.863
Iteration  3  Error:     0.223

Align patch 24  1 left
Iteration  1  Error:     3.992
Iteration  2  Error:     0.737
Iteration  3  Error:     0.159

Align patch 25  0 left
Iteration  1  Error:     2.868
Iteration  2  Error:     0.526
Iteration  3  Error:     0.162

3

Align patch 5  20 left
Iteration  1  Error:     1.011
Iteration  2  Error:     0.276

Align patch 6  19 left
Iteration  1  Error:     1.077
Iteration  2  Error:     0.151

Align patch 7  18 left
Iteration  1  Error:     1.204
Iteration  2  Error:     0.162

Align patch 8  17 left
Iteration  1  Error:     1.272
Iteration  2  Error:     0.105

Align patch 9  16 left
Iteration  1  Error:     1.943
Iteration  2  Error:     0.382

Align patch 10  15 left
Iteration  1  Error:     2.766
Iteration  2  Error:     0.289

Align patch 11  14 left
Iteration  1  Error:     2.280
Iteration  2  Error:     0.687
Iteration  3  Error:     0.201

Align patch 12  13 left
Iteration  1  Error:     1.295
Iteration  2  Error:     0.186

Align patch 13  12 left
Iteration  1  Error:     1.601
Iteration  2  Error:     0.101

Align patch 14  11 left
Iteration  1  Error:     1.076
Iteration  2  Error:     0.127

Align patch 15  10 left
Iteration  1  Error:     3.421
Iteration  2  Error:     1.052
Iteration  3  Erro

## Remove `tmp` Directory

In [14]:
os.removedirs('tmp')